# Part I. ETL Pipeline for Pre-Processing the Files

## THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [20]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [21]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


---

# Part II. Apache Cassandra coding. 

## Now we are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1']) # This should make a connection to a Cassandra instance your local machine 
    session = cluster.connect() # To establish connection and begin executing queries, need a session
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS ahmaddb
    WITH REPLICATION =
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
                   )
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('ahmaddb')
except Exception as e:
    print(e)

---

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




---

### Query 1: Give me the artist, song title and song's length in the music app history that was heard during \ 
### sessionId = 338, and itemInSession = 4

#### Creating the table

In [8]:
query = "CREATE TABLE IF NOT EXISTS songplay "
query = query + "(artist text, song text, length float, sessionId int, itemInSession int, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

#### Insert the data into the table

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songplay (artist, song, length, sessionId, itemInSession)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

#### SELECT statement to view the result of query


In [10]:
query = "SELECT artist, song, length from songplay where sessionId = 338 AND itemInSession = 4 "

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# To print the query result
for row in rows:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


---

### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) \
### for userid = 10, sessionid = 182

#### Creating the table

In [11]:
query = "CREATE TABLE IF NOT EXISTS music_app_history "
query = query + "(artist text, song text, itemInSession int, firstName text, lastName text, userId int, sessionId int, PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

#### Insert the data into the table

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_app_history (artist, song, itemInSession, firstName, lastName, userId, sessionId)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], int(line[3]), line[1], line[4], int(line[-1]), int(line[-3])))

#### SELECT statement to view the result of query

In [19]:
query = "SELECT * from music_app_history WHERE userId = 10 AND sessionId = 182 "

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# print the query result
for row in rows: 
    print("Artist:",row.artist," FirstName:", row.firstname," LastName:", row.lastname," Song:", row.song)

Artist: Down To The Bone  FirstName: Sylvie  LastName: Cruz  Song: Keep On Keepin' On
Artist: Three Drives  FirstName: Sylvie  LastName: Cruz  Song: Greece 2000
Artist: Sebastien Tellier  FirstName: Sylvie  LastName: Cruz  Song: Kilometer
Artist: Lonnie Gordon  FirstName: Sylvie  LastName: Cruz  Song: Catch You Baby (Steve Pitron & Max Sanna Radio Edit)


---

### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### Creating the table

In [14]:
query = "CREATE TABLE IF NOT EXISTS music_history "
query = query + "(song text, firstName text, lastName text, userId int, PRIMARY KEY (song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

#### Insert the data into the table

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_history (song, firstName, lastName, userId)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], line[1], line[4], int(line[-1])))

#### SELECT statement to view the result of query

In [16]:
query = "SELECT firstName, lastName from music_history where song='All Hands Against His Own' "

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# To print the query result
for row in rows: 
    print(row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

Drop the table before closing out the sessions

In [18]:
# DROP table 1
session.execute("DROP TABLE IF EXISTS songplay")

In [19]:
# DROP table 2
session.execute("DROP TABLE IF EXISTS music_app_history")

In [20]:
# DROP table 3
session.execute("DROP TABLE IF EXISTS music_history")

### Close the session and cluster connection¶

In [21]:
session.shutdown()
cluster.shutdown()